In [1]:
import pandas as pd #reading csv fiel
import numpy as np #array manupulation
import matplotlib.pyplot as plt #plotting figures
from keras.models import Sequential
from keras.layers import BatchNormalization, Conv2D, Activation, MaxPooling2D, Dense, GlobalAveragePooling2D, Dropout, Flatten
from keras import optimizers
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MinMaxScaler
from tqdm import tqdm

Using TensorFlow backend.


In [2]:
train = pd.read_csv("fer2013/fer2013.csv")

In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35887 entries, 0 to 35886
Data columns (total 3 columns):
emotion    35887 non-null int64
pixels     35887 non-null object
Usage      35887 non-null object
dtypes: int64(1), object(2)
memory usage: 841.2+ KB


In [4]:
type(train.pixels[0])

str

In [5]:
#pixel values are of type string have to coemverte em to integet before reshaping

In [30]:
from PIL import Image

def get_numpy_array(data):
    data_image = data.reshape((48, 48))
    data_image = Image.fromarray(data_image).convert('L')
    data_image = np.array(data_image)[:, :].copy()
    return data_image.reshape(48,48,1)

def get_data(train_Data):
    images = []
    emotion = []
    for inedx, row in tqdm(train_Data.iterrows()):
        split_data = np.fromstring(str(row.pixels), dtype=np.uint8, sep=' ')
        imagearray = get_numpy_array(split_data)
        imagearray = imagearray/255
        images.append(imagearray)
        emotion.append(row.emotion)
    return np.array(emotion), np.array(images)

In [31]:
emotions, images = get_data(train)

35887it [00:08, 4071.48it/s]


In [32]:
images.shape

(35887, 48, 48, 1)

In [19]:
def imageConverter(string):
    return np.array([int(item) for item in string.split(' ')]).reshape((48, 48))

def GetAllImages():
    imagearray=[]
    for imageString in train['pixels']:
        image = imageConverter(imageString)
        imagearray.append(image)
    return np.array(imagearray)

In [21]:
X = np.stack([imageConverter(string) for string in train['pixels']]).astype(np.float)[:, :, :, np.newaxis]

In [18]:
X_train =images
X_train.shape

(35887, 48, 48, 1)

In [23]:
print("Shape image array = ",X_train.shape)
print("shape of emotion array", emotions.shape)
emotions = emotions.reshape((emotions.shape[0],1))
print("shape of emotion array", emotions.shape)
print(np.unique(emotions))

Shape image array =  (35887, 48, 48, 1)
shape of emotion array (35887, 1)
shape of emotion array (35887, 1)
[0 1 2 3 4 5 6]


In [19]:
import matplotlib.pyplot as plt

In [20]:
from sklearn.preprocessing import OneHotEncoder

onehot_encoder = OneHotEncoder(sparse=False)
emotions = emotions.reshape(len(emotions), 1)
onehot_encoded = onehot_encoder.fit_transform(emotions)
print(onehot_encoded[0])

[1. 0. 0. 0. 0. 0. 0.]


In [35]:
model = Sequential()
# input layer
model.add(BatchNormalization(input_shape=(48, 48,1)))
model.add(Conv2D(12, (2,2), kernel_initializer='he_normal'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.1))
# layer 2
model.add(Conv2D(24, (2, 2)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.2))


# layer 3
model.add(Conv2D(48, (2, 2)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.3))


# layer 4
model.add(Conv2D(96, (2, 2)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.4))


# layer 5
model.add(Conv2D(192, (2, 2)))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Flatten())

# layer 8
model.add(Dense(1000, activation="relu"))
model.add(Dropout(0.5))
# layer 9
model.add(Dense(1000, activation="relu"))

# layer 10
model.add(Dense(7))

In [ ]:
sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
epochs = 20
history = model.fit(X_train, onehot_encoded, validation_split=0.3, shuffle=True, epochs=epochs)

Train on 25120 samples, validate on 10767 samples
Epoch 1/20
25120/25120 [==============================] - 43s 2ms/step - loss: 0.1168 - acc: 0.2628 - val_loss: 0.1140 - val_acc: 0.2909
Epoch 2/20
25120/25120 [==============================] - 42s 2ms/step - loss: 0.1144 - acc: 0.2928 - val_loss: 0.1130 - val_acc: 0.3000
Epoch 3/20
25120/25120 [==============================] - 41s 2ms/step - loss: 0.1125 - acc: 0.3068 - val_loss: 0.1078 - val_acc: 0.3369
Epoch 4/20
25120/25120 [==============================] - 55s 2ms/step - loss: 0.1095 - acc: 0.3356 - val_loss: 0.1031 - val_acc: 0.3798
Epoch 5/20
25120/25120 [==============================] - 56s 2ms/step - loss: 0.1061 - acc: 0.3643 - val_loss: 0.0993 - val_acc: 0.4271
Epoch 6/20
25120/25120 [==============================] - 49s 2ms/step - loss: 0.1035 - acc: 0.3862 - val_loss: 0.0968 - val_acc: 0.4464
Epoch 7/20
25120/25120 [==============================] - 46s 2ms/step - loss: 0.1019 - acc: 0.3984 - val_loss: 0.0968 - val_acc

In [ ]:
model.save("Emotion_analysis")